In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.python.keras.models import load_model

2023-02-28 09:10:07.539943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 09:10:08.167947: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 09:10:08.167967: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-28 09:10:10.110557: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Get test data

In [2]:
train_df = pd.read_csv('training_6_category_1/preprocessed.csv')
train_df['type_sensor_1'] = train_df['type_sensor_1'].astype('category')
train_df['type_sensor_2'] = train_df['type_sensor_2'].astype('category')
train_df['type_sensor_3'] = train_df['type_sensor_3'].astype('category')

X = train_df.drop(columns=['latitude_aircraft', 'longitude_aircraft', 'geoAltitude', 'baroAltitude'])
Y = train_df[['latitude_aircraft', 'longitude_aircraft', 'geoAltitude', 'baroAltitude']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# SKLearn

Evaluate different models from SKLearn

## Multi Output Regression

First model is a simple multi output regression.
Multi output regression because we predict longitude and latitude.
Multi output regression does not learn the relation between the two predicted variables.

In [3]:
with open('models/mlr_model_aircraft_pred_no_alt.pkl', 'rb') as mlr_fh:
    mlr = pickle.load(mlr_fh)

pred = mlr.predict(X_test)
print('Linear Regression')
print(f"r2 long: {r2_score(y_test['longitude_aircraft'], [long for long, lat in pred])}")
print(f"r2 lat: {r2_score(y_test['latitude_aircraft'], [lat for long, lat in pred])}")
print(f"rmse long: {mean_squared_error(y_test['longitude_aircraft'], [long for long, lat in pred], squared=False)}")
print(f"rmse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for long, lat in pred], squared=False)}")
print(f"mse long: {mean_squared_error(y_test['longitude_aircraft'], [long for long, lat in pred])}")
print(f"mse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for long, lat in pred])}\n\n")

Linear Regression
r2 long: 0.0008520497230456447
r2 lat: 0.0008016530933001587
rmse long: 4.201904586667343
rmse lat: 3.0937101522284562
mse long: 17.656002155456058
mse lat: 9.571042506001419




## Multitask Elastic Net without Altitude

In [4]:
with open('models/men_model_aircraft_pred_no_alt.pkl', 'rb') as men_no_alt_fh:
    men_no_alt = pickle.load(men_no_alt_fh)

pred = men_no_alt.predict(X_test)

print('Multitask Elastic Net without Altitude')
print(f"r2 long: {r2_score(y_test['longitude_aircraft'], [long for long, lat in pred])}")
print(f"r2 lat: {r2_score(y_test['latitude_aircraft'], [lat for long, lat in pred])}")
print(f"rmse long: {mean_squared_error(y_test['longitude_aircraft'], [long for long, lat in pred], squared=False)}")
print(f"rmse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for long, lat in pred], squared=False)}")
print(f"mse long: {mean_squared_error(y_test['longitude_aircraft'], [long for long, lat in pred])}")
print(f"mse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for long, lat in pred])}\n\n")

Multitask Elastic Net without Altitude
r2 long: -1.3774966922674992e-07
r2 lat: -2.3311487105104334e-06
rmse long: 4.203696136779482
rmse lat: 3.0949545468389936
mse long: 17.671061210374738
mse lat: 9.57874364699936




## Multitask Elastic Net with Altitude

In [5]:
with open('models/men_model_aircraft_pred.pkl', 'rb') as men_fh:
    men = pickle.load(men_fh)

pred = men.predict(X_test)

print('Multitask Elastic Net with Altitude')
print(f"r2 long: {r2_score(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred])}")
print(f"r2 lat: {r2_score(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred])}")
print(f"rmse long: {mean_squared_error(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred], squared=False)}")
print(f"rmse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred], squared=False)}")
print(f"mse long: {mean_squared_error(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred])}")
print(f"mse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred])}\n\n")

Multitask Elastic Net with Altitude
r2 long: 1.0670027573933538e-05
r2 lat: 1.7049724647666409e-06
rmse long: 4.20367342041551
rmse lat: 3.09494830104146
mse long: 17.670870225507834
mse lat: 9.57870498611942




## Multitask Elastic Net with Altitude and optimized L1 ratio

In [6]:
with open('models/men_model_aircraft_pred_l1.pkl', 'rb') as men_fh:
    men = pickle.load(men_fh)

pred = men.predict(X_test)

print('Multitask Elastic Net with Altitude and Optimized L1 Ratio')
print(f"r2 long: {r2_score(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred])}")
print(f"r2 lat: {r2_score(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred])}")
print(f"rmse long: {mean_squared_error(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred], squared=False)}")
print(f"rmse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred], squared=False)}")
print(f"mse long: {mean_squared_error(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred])}")
print(f"mse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred])}\n\n")

Multitask Elastic Net with Altitude and Optimized L1 Ratio
r2 long: 5.712066375118674e-05
r2 lat: 0.0002136923400199997
rmse long: 4.203575786587623
rmse lat: 3.094620238123373
mse long: 17.670049393585757
mse lat: 9.57667441820276




# Tensorflow

In [10]:
nn_model = load_model('models/best_aircraft_model200.hdf5')
pred = nn_model.predict(X_test)

print('Neural Net')
print(f"r2 long: {r2_score(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred])}")
print(f"r2 lat: {r2_score(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred])}")
print(f"rmse long: {mean_squared_error(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred], squared=False)}")
print(f"rmse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred], squared=False)}")
print(f"mse long: {mean_squared_error(y_test['longitude_aircraft'], [long for lat, long, geo, baro  in pred])}")
print(f"mse lat: {mean_squared_error(y_test['latitude_aircraft'], [lat for lat, long, geo, baro  in pred])}\n\n")

2023-02-28 09:14:46.814363: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 88558720 exceeds 10% of free system memory.


Neural Net
r2 long: -0.0019075471298186653
r2 lat: -0.04839995220555249
rmse long: 4.207703311028668
rmse lat: 3.168963706251063
mse long: 17.70476715364162
mse lat: 10.042330971536474


